In [36]:
from curl_cffi import requests
from bs4 import BeautifulSoup
import re
import time
import gen_css
import add_current
import BaiduTrans

In [7]:
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

In [8]:
jle = requests.get('https://www.journals.uchicago.edu/toc/jole/current',impersonate = 'chrome110',headers=headers)
jle

<Response [200]>

In [9]:
jle_data = BeautifulSoup(jle.text, 'html.parser')
jle_data


<!DOCTYPE html>

<html class="pb-page" data-request-id="02822803c4604832b273f7575ba2392e" lang="en">
<head data-pb-dropzone="head">
<meta charset="utf-8"/>
<title>Journal of Labor Economics | Vol 42, No 3</title>
<link href="/wro/nd03~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/uchicago/releasedAssets/images/favicon/favicon-35991e76163d690e92a3ff7033d8f50c.ico" rel="icon" type="image/x-icon">
<link href="/products/uchicago/releasedAssets/css/build-c7664c1ace15d4eabc21.css" id="build-style-sheet" media="all" rel="stylesheet"/>
<link href="/products/uchicago/releasedAssets/css/print-c7664c1ace15d4eabc21.css" media="print" rel="stylesheet"/>
<script nonce="02822803c4604832b273f7575ba2392e">
        try {
            document.documentElement.classList.add('js');
        } catch (e) {
        }
    </script>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/pb-assets/styles/custom-1709913767723.css" rel="stylesheet" type="text/css"/>
<!-- Global 

In [10]:
current_lin1 = jle_data.find(class_="cover-date").text
current_lin1 = current_lin1[-4:]
current_lin1

'2024'

In [11]:
current_lin2 = jle_data.find(class_="citation-line").text
match = re.search(r'Volume (\d+), Number (\d+)',current_lin2)
volume = match.group(1)
number = match.group(2)
current_lin2 = f'{volume}/{number}'
current_lin2

'42/3'

In [12]:
current_link = current_lin1+'/'+current_lin2
current_link

'2024/42/3'

In [13]:
current_url = 'https://www.journals.uchicago.edu/toc/jole/' + current_link
current_url

'https://www.journals.uchicago.edu/toc/jole/2024/42/3'

In [14]:
title_of_c = jle_data.find('title').text
title_of_c

'Journal of Labor Economics | Vol 42, No 3'

In [16]:
issue = re.search('Vol.+',title_of_c)[0]
issue = re.sub(r"/", "_", issue)
issue

'Vol 42, No 3'

In [17]:
current_data = requests.get(current_url,impersonate = 'chrome110',headers=headers)
current_data

<Response [200]>

In [18]:
data = BeautifulSoup(current_data.text, 'html.parser')
data


<!DOCTYPE html>

<html class="pb-page" data-request-id="5ba889bfe66a44b7bd2772eca0748873" lang="en">
<head data-pb-dropzone="head">
<meta charset="utf-8"/>
<title>Journal of Labor Economics | Vol 42, No 3</title>
<link href="/wro/nd03~product.css" rel="stylesheet" type="text/css"/>
<link href="/products/uchicago/releasedAssets/images/favicon/favicon-35991e76163d690e92a3ff7033d8f50c.ico" rel="icon" type="image/x-icon">
<link href="/products/uchicago/releasedAssets/css/build-c7664c1ace15d4eabc21.css" id="build-style-sheet" media="all" rel="stylesheet"/>
<link href="/products/uchicago/releasedAssets/css/print-c7664c1ace15d4eabc21.css" media="print" rel="stylesheet"/>
<script nonce="5ba889bfe66a44b7bd2772eca0748873">
        try {
            document.documentElement.classList.add('js');
        } catch (e) {
        }
    </script>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/pb-assets/styles/custom-1709913767723.css" rel="stylesheet" type="text/css"/>
<!-- Global 

In [19]:
articles = data.find_all(class_='issue-item__title')
articles

[<h4 class="issue-item__title"><a href="/doi/10.1086/731817" id="d487015e389a1051">Front Matter</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/731666" id="d487015e434a1051">SOLE 2025/30th Annual Meeting: Call for Papers</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/731667" id="d487015e488a1051">Officers of the Society</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/724157" id="d487015e535a1051">The Effect of Job Search Requirements on Family Welfare Receipt</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/724048" id="d487015e645a1051">Grades and Employer Learning</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/724570" id="d487015e746a1051">The Decline in Rent Sharing</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/724568" id="d487015e847a1051">Who Benefits from Attending Effective High Schools?</a></h4>,
 <h4 class="issue-item__title"><a href="/doi/10.1086/725001" id="d487015e956a1051">School Pe

In [28]:
title = []
name = []
abstracts = []
for art_link in articles:
    try:
        art_link = 'https://www.journals.uchicago.edu'+ art_link.find('a').get('href')
        article_requests = requests.get(art_link,impersonate = 'chrome110',headers=headers)
        article_data = BeautifulSoup(article_requests.text, 'html.parser')
        abstract = article_data.find(class_='abstractSection abstractInFull').get_text()
        title.append(article_data.find(class_='citation__title').text)
        ori_name = article_data.find_all(class_='author-name accordion-tabbed__control visible-x')
        names = ""
        for span in ori_name:
            names += span.get_text(strip=True) + ", "
            names = names[:-2]
        name.append(names)
        abstracts.append(abstract)
       
    except:
        pass
article = [*zip(title, name, abstracts)]
article

[('The Effect of Job Search Requirements on Family Welfare Receipt',
  'Marc K. ChanNicolas HéraultHa VuRoger Wilkins',
  'Many countries impose job search requirements on unemployment benefit recipients. Existing studies have mainly focused on the impact of incremental changes to requirements on the recipients. Australian reforms in 1995 saw groups of partnered welfare recipients newly subjected to job search requirements, allowing us to produce causal estimates of the effects on family welfare receipt of introducing such requirements. Using a quasi-experimental design and administrative data, we find large negative effects on welfare receipt for the mature-age partnered women targeted by the reforms and similar effects on welfare receipt of their partners, suggesting that family labor supply decisions were considerably affected.'),
 ('The Decline in Rent Sharing',
  'Brian BellPaweł BukowskiStephen Machin',
  'The evolution of rent sharing is studied. Based on a panel of the top 300 

In [32]:
file_folder = 'jle/'
jounal_name = 'JLE_'

In [34]:
with open(file_folder + jounal_name + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title))
    f.write(gen_css.gen_abstract_list(list(article)))
add_current.add(jounal_name + issue, time.ctime(), file_folder + jounal_name + issue + '.html')

In [37]:
abstract_cn = BaiduTrans.trans_list(abstracts)
title_cn = BaiduTrans.trans_list(title)
article_cn = [*zip(title_cn, name, abstract_cn)]
with open(file_folder + jounal_name + 'CN_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(title_cn))
    f.write(gen_css.gen_abstract_list(list(article_cn)))
add_current.add(jounal_name + 'CN_' + issue, time.ctime(), file_folder + jounal_name + 'CN_' + issue + '.html')